In [1]:
import numpy as np
from astropy.table import Table
from astropy.table import vstack, hstack
from matplotlib import pyplot as plt
from astropy.io import ascii
import halotools.utils as halotools
import math as math

In [2]:
class constant:
    ### defining some constants
    ## graviational constant in cgs
    G = 6.67 * 10**-8.0
    ## Omega baryon
    Omega_b = 0.046
    ### cosmic baryon fraction
    f_baryon = Omega_b/0.279
    ## virial ratio
    xi_vir = 178.0
    # z=0 average density in M_solar/Mpc^3
    rho_o = 2.0*10**10.0

In [3]:
class parameter:
    star_popIII = ['mf']
    alpha = [0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9,2.1,2.35]
    f_kinetic = [0.3]
    sfe_III = [0.01]

    #extend full range from 0.1 - 5%
        
    #sf_metal_III = 0.7

    f_crit = 0.02*10**-5
    ESN_popIII = [10.0**51.0]
    sfe_II = 0.01
    Nfrag = 6.0
    
    III_max = [300.0]
    III_min = [10.0]
    
    PISN_min = [140.0]
    
    WDM_mass = 10**0.0
    
    boxsize = 2.0
    


In [4]:
def f_coll(a):
        M_collapse = np.sum(tree['mvir'][tree['scale']==a].quantity) 
        return  M_collapse/(constant.rho_o*parameter.boxsize**3.0)
    
def rho_IGM(a):
        return constant.rho_o*(1-f_coll(a))*a**-3.0

In [5]:
class threshold:
    def M_HI(scale):
        return 7.75 * 10.0**6.0 * (31.0*scale)**1.5
    M_HI = np.vectorize(M_HI)
  

    def M_III(scale):
        M_jeans = 1.905 * 10.0**5.0 * (31.0*scale)**1.5
        M_H2 = 1.36 * 10.0**5.0 * (31.0*scale)**2.071
        return np.maximum(M_jeans,M_H2)
    
    M_III = np.vectorize(M_III)

class externalJ21:
    def TS09_high(scale):
        if((1.0/scale-1.0)>60):
            J_21 = 0
        else:
            J_21 = 10.0**(-0.106*((1.0/scale - 1.0)-10.0) + 0.78)
        return J_21
    
    TS09_high = np.vectorize(TS09_high)

    def TS09_medium(scale):
        if((1.0/scale-1.0)>39.5):
            J_21 = 0
        else:
            J_21 = 10.0**(-0.078*((1.0/scale - 1.0)-10.0) - 0.4)
        return J_21
    
    TS09_medium = np.vectorize(TS09_medium)
    
    def TS09_low(scale):
        if((1.0/scale-1.0)>31.5):
            J_21 = 0
        else:
            J_21 = 10.0**(-0.04*((1.0/scale-1.0)-10.0) - 2.3)
    
        return J_21
    
    TS09_low = np.vectorize(TS09_low)

    
    def J21_constant(scale,const):
        J_21 = const
        
        return J_21

    J21_constant = np.vectorize(J21_constant)
    


In [6]:
## self consistent LW calculation
class J21_internal:
    def nLW_PopIII(i,scale,N_ph_III,mask_PIII):
        n=0.0
        f_LW = 1.0
        #n = sum(N_ph_III[(scale<=scale[i]) & (scale>11.18/12.39*scale[i])])
        n = N_ph_III[i] = 8.0 * np.sum(tree['III_new'][(tree['scale']<=scale[i]) & (tree['scale']>11.18/12.39*scale[i]) & (tree['III_new']>0.0)].quantity)
        n = n*(parameter.boxsize)**-3.0
        
        return n
    
    def J21_PopIII(i,a,nLW_PopIII):
        J_21 = 1.6 * 10.0**-5.0 * (nLW_PopIII[i])*(31.0*a)**-3.0
        return J_21
    
    def nLW_PopII(i,scale,N_ph_II,mask_PII):
        f_LW = 1.0
        a = scale[i]
        # summing total number of photons
        #N_ph_II[i] = f_LW*np.sum(0.8 * tree['II_new'][mask_PII])
        # calculate n_LW in Mpc^-3
        n = N_ph_II[i] = 8.0 * np.sum(tree['m_popII_LW'][(tree['scale']<=scale[i]) & (tree['scale']>11.18/12.39*scale[i]) & (tree['m_popII_LW']>0.0)].quantity)
        n = n*(parameter.boxsize)**-3.0

        return n
        
    def J21_PopII(i,a,nLW_PopII):
        J_21 = 1.6 * 10.0**-5.0 * (nLW_PopII[i])*(31.0*a)**-3.0

        
        return J_21

In [7]:

def PopIII_IMF(value,alpha,mask_PIII):

    if value==150:
        # this is using the orignial fragmentation equation which we believe may be wrong.
        #N_frag = parameter.Nfrag[nfrag]*(np.trunc(3.02* np.array((tree['mvir'][mask_PIII]/10**6.0)**(2.0/3.0)*1.0/(tree['scale'][mask_PIII]*31.0)))+1)
        N_possible = np.trunc(parameter.sfe_III[sfe]*constant.f_baryon*tree['mvir'][mask_PIII]/parameter.star_popIII[imf])
        N_frag[N_possible<N_frag]=N_possible[N_possible<N_frag]
        
        tree['N_CCSN'][mask_PIII] = 0
        tree['N_PISN'][mask_PIII] = N_frag
        tree['N_DCBH'][mask_PIII] = 0
    
        tree['E_SN'][mask_PIII] = tree['N_PISN'][mask_PIII]*parameter.f_kinetic[fkin]*10.0**53.0
        tree['M_metal'][mask_PIII] = 0.7*tree['N_PISN'][mask_PIII]*parameter.star_popIII[imf]

        return N_frag*parameter.star_popIII[imf]
    
    elif value==40:
        # this is using the orignial fragmentation equation which we believe may be wrong.
        #N_frag = np.trunc(3.02 * np.array((tree['mvir'][mask_PIII]/10**6.0)**(2.0/3.0)*1.0/(tree['scale'][mask_PIII]*31.0))+1)
        N_possible = np.trunc(parameter.sfe_III[sfe]*constant.f_baryon*tree['mvir'][mask_PIII]/parameter.star_popIII[imf])
        N_frag[N_possible<N_frag]=N_possible[N_possible<N_frag]


        tree['N_CCSN'][mask_PIII] = N_frag
        tree['N_PISN'][mask_PIII] = 0
        tree['N_DCBH'][mask_PIII] = 0
    
        tree['E_SN'][mask_PIII] = N_frag*parameter.f_kinetic[fkin]*10.0**51.0
        tree['M_metal'][mask_PIII] = 0.2*N_frag*parameter.star_popIII[imf]

        return N_frag*parameter.star_popIII[imf]

    elif value=='single':
        
        N_frag = 1.0

        tree['N_CCSN'][mask_PIII] = 0
        tree['N_PISN'][mask_PIII] = N_frag
        tree['N_DCBH'][mask_PIII] = 0
    
        tree['E_SN'][mask_PIII] = N_frag*parameter.f_kinetic[fkin]*10.0**53.0
        tree['M_metal'][mask_PIII] = 0.7*N_frag*150.0

        return 150.0


    elif value=='mf': 
        print("Pop III power law IMF")
        
        # this is using the orignial fragmentation equation which we believe may be wrong.
        #N_frag = np.trunc(3.02 * np.array((tree['mvir'][mask_PIII]/10**6.0)**(2.0/3.0)*1.0/(tree['scale'][mask_PIII]*31.0)))+1
        #updated equation with the 3/4 instead of 3/2 exponent.
        N_frag = np.trunc(9.12 * np.array((tree['mvir'][mask_PIII]/10**6.0)**(4.0/3.0)*(1.0/(tree['scale'][mask_PIII]*31.0))**2.0))+1


        alpha_III = alpha
        
        M_stars = np.zeros(len(N_frag))
        N_C = np.zeros(len(N_frag))
        N_P = np.zeros(len(N_frag))
        N_B = np.zeros(len(N_frag))
        NIII = np.zeros(len(N_frag))
        M_CCSN = np.zeros(len(N_frag))
        M_PISN = np.zeros(len(N_frag))

        for i in range(0,len(N_frag)):
            N_III = 0

            while((N_III<1)):
                if(alpha_III<0.0):
                    alpha_III=-alpha_III
                    s = np.random.power(1.0+alpha_III,100*np.int(N_frag[i]))
                    mass = parameter.III_min * (1.0-s) ** (-1.0/alpha_III)
                
                    mass = mass[mass<parameter.III_max]

                else:
                    s = np.random.power(1.0+alpha_III,1000)
                    mass = parameter.III_max * (1.0-s) ** (1.0/alpha_III)
                
                    mass = mass[mass>parameter.III_min]

                M_tot  = np.cumsum(mass)
                
                mask_stars = M_tot<=parameter.sfe_III[sfe]*constant.f_baryon*tree['mvir'][mask_PIII][i]
                
                M_III = mass[mask_stars]
                N_III = len(M_III)
                ### cutting to the maximum possible number of stars
                if(N_III>parameter.Nfrag*N_frag[i]):
                    M_III = M_III[:int(parameter.Nfrag*N_frag[i])]
            
            N_C[i] = len(M_III[(M_III>8.0) & (M_III<parameter.PISN_min[PISN])])
            M_CCSN[i] = np.sum(M_III[np.where((M_III>8.0) & (M_III<parameter.PISN_min[PISN]))])
            N_P[i] = len(M_III[np.where((M_III>parameter.PISN_min[PISN]) & (M_III<260.0))])
            M_PISN[i] = np.sum(M_III[np.where((M_III>parameter.PISN_min[PISN]) & (M_III<260.0))])
            N_B[i] =  len(M_III[np.where(M_III>260.0)])
                     
            NIII[i] = len(M_III)
            
            M_stars[i] = np.sum(M_III)

        tree['N_CCSN'][mask_PIII] = N_C
        tree['N_PISN'][mask_PIII] = N_P
        tree['N_DCBH'][mask_PIII] = N_B
        tree['E_SN'][mask_PIII] = tree['N_PISN'][mask_PIII]*parameter.f_kinetic[fkin]*10.0**53.0 + tree['N_CCSN'][mask_PIII]*parameter.f_kinetic[fkin]*10.0**51.0
        tree['M_metal'][mask_PIII] = 0.7*M_PISN + 0.2*M_CCSN
        tree['N_III'][mask_PIII] = NIII
        
        return M_stars


In [8]:
def Renrich(a):
        
        halos = tree[tree['scale']==a]
        R_en = np.zeros(len(halos['rvir']))
        
        M_contain = 7.96*10**6.0*(halos['E_SN']/10**51.0)**0.6*(31.0*a)**0.6
        #### rho_IGM*10**-9.0 is in M_solar/kpc^3s
        
        a = 3.0/(4.0*math.pi*rho_IGM(a)*10**-9.0)
        for i in range(0,len(halos['M_metal'])):
            if((halos['M_metal'][i]>0.0) & (halos['mvir'][i]<M_contain[i])):
                b = halos['M_metal'][i]/(parameter.f_crit * constant.f_baryon)
                ##### virial radius is in comoving kpc
                R_en[i] = a*(b - halos['mvir'][i]) + (halos['rvir'][i]*halos['scale'][i])**3.0
        return R_en**(1.0/3.0)
    

def Rejecta(a):
        
        halos = tree[tree['scale']==a]
        M_contain = 7.96*10.0**6.0*(halos['E_SN']/10.0**51.0)**0.6*(31.0*a)**0.6
                
        Mvir= halos['mvir']/0.7
        Rvir = halos['rvir']*halos['scale']/0.7
                
        #### check units here.....
        E_SN = halos['E_SN']/(18.0*10.0**75.0)
        G = constant.G/(0.135*10.0**32.0)
        
        R_ej = np.zeros(len(halos['rvir']))
        
        C_0_a = Mvir**2.0
        ##### virial radius is in comoving kpc
        C_0_b = Mvir*rho_IGM(a)*10.0**-9.0*2.0*(4.0*math.pi/3.0)*Rvir**3.0
        #### virial radius is in comoving kpc
        C_0_c = (rho_IGM(a)*10.0**-9.0)*(4.0*math.pi/3.0)**2.0*Rvir**6.0
        C_0 = C_0_a - C_0_b + C_0_c
        
        C_1 = -E_SN/(3.0*constant.f_baryon*G)
        #### converstion from g**2.0/cm to M_solar**2.0/kpc
        
        C_2 = 0.0
        
        C_3_a = 2.0*(4.0*math.pi/3.0)*Mvir*rho_IGM(a)*10.0**-9.0
        ##### virial radius is in comoving kpc
        C_3_b = 2.0*(rho_IGM(a)*10.0**-9.0)**2.0*(4*math.pi/3.0)**2.0*Rvir**3.0
        C_3 = C_3_a - C_3_b
        
        C_4 = 0.0
        
        C_5 = 0.0
        
        C_6 = (4.0*math.pi/3.0)**2.0*(rho_IGM(a)*10**-9.0)**2.0
        
        for i in range(0,len(C_0)):
            if((halos['mvir'][i]<M_contain[i])):
                
                def f(x):
                    y = C_6*x**6.0 + C_5*x**5.0 + C_4*x**4.0 + C_3[i]*x**3.0 + C_2*x**2.0 + C_1[i]*x + C_0[i]
                    return y
                
                x0 = np.polynomial.polynomial.polyroots([C_0[i],C_1[i],C_2,C_3[i],C_4,C_5,C_6])
                
                if len(np.real(x0[np.isreal(x0)])[np.where(np.real(x0[np.isreal(x0)])>0.0)])>0:
                    R_ej[i] = np.max(np.real(x0[np.isreal(x0)])[np.where(np.real(x0[np.isreal(x0)])>0.0)])        

        return R_ej

In [9]:
def metal_pollution_Nfrag():    
    N_PopIII = parameter.Nfrag*np.trunc(3.02 * np.array((tree['mvir'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])]/10**6.0)**(2.0/3.0)*1.0/(tree['scale'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])]*31.0)))+1
    
    M_LW[0] = 6.44*10.0**6.0 * (J21_ext[0])**0.457 * ((1.0/scale[0])/31.0)**-3.55
    M_PIII[0] = np.maximum(threshold.M_III(scale[0]),M_LW[0])
    M_PIII[0] = np.minimum(threshold.M_HI(scale[0]),M_PIII[0])

    #mask = np.less(N_PopIII*parameter.star_popIII[imf], parameter.sfe_III[sfe]*constant.f_baryon*np.array(tree['mvir'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])]))
    #N_gas = np.trunc(parameter.sfe_III[sfe]*constant.f_baryon*np.array(tree['mvir'][(tree['scale']==scale[0]) & (tree['mvir']> M_PIII[0])])/parameter.star_popIII[imf])
    #N_PopIII[mask] = N_gas[mask]
    
        
    ### have PopIII_IMF switch return the total mass in m_popIII stars
    mask_PIII = (tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])
    tree['m_popIII'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])] = PopIII_IMF(parameter.star_popIII[imf],parameter.alpha[a],mask_PIII) 
    tree['polluted'][(tree['scale']==scale[0]) & (tree['m_popIII'] > 0.0)] = 1
    tree['enriched'][(tree['scale']==scale[0]) & (tree['m_popIII'] > 0.0)] = 1
    tree['III_new'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])] = tree['m_popIII'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PIII[0])]
    tree['m_popII'][(tree['scale']==scale[0]) & (tree['mvir'] > M_PII[0])] = parameter.sfe_II*constant.f_baryon*tree['mvir'][(tree['scale'] == scale[0]) & (tree['mvir'] > M_PII[0])]
    tree['R_ejecta'][tree['scale']==scale[0]] = Rejecta(scale[0])
    
    M_PII[0] = np.minimum(threshold.M_HI(scale[0]),M_LW[0])
    mask_PII = (tree['scale']==scale[0]) & (tree['mvir'] > M_PII[0])& ((tree['polluted']!=0) | (tree['enriched']!=0)) & (tree['III_new']==0.0)

    
    for i in range(1,len(scale)):
        # calcuating the self consistent LW generated in the previous snapshot
        nLW_PopIII[i] = J21_internal.nLW_PopIII(i,scale,nLW_PopIII,mask_PIII)
        J21_int_PopIII[i] = J21_internal.J21_PopIII(i,scale[i],nLW_PopIII)
        nLW_PopII[i] = J21_internal.nLW_PopII(i,scale,nLW_PopII,mask_PII)
        J21_int_PopII[i] = J21_internal.J21_PopII(i,scale[i],nLW_PopII)
   
        ids = tree['id'][tree['scale']==scale[i]]
    
        #### grouping halos in previous snapshot by descendent ID
        halos = tree[tree['scale']==scale[i-1]]
    
        halos.sort('desc_id')
    
        ### determining which indexes in halo arrays are unique
        d_id,mask = np.unique(halos['desc_id'],return_index=True)
        halo_unique = halos[mask]
    
        grouping_key = 'desc_id'
        requested_columns = ['m_popIII','m_popII','M_metal','polluted','enriched','N_DCBH']
        group_gen = halotools.group_member_generator(halos, grouping_key, requested_columns)
        
        result_III = np.zeros(len(tree[tree['scale']==scale[i-1]]))
        result_II = np.zeros(len(tree[tree['scale']==scale[i-1]]))
        result_metal = np.zeros(len(tree[tree['scale']==scale[i-1]]))
        result_poll = np.zeros(len(tree[tree['scale']==scale[i-1]]))
        result_enrich = np.zeros(len(tree[tree['scale']==scale[i-1]]))
        result_DCBH = np.zeros(len(tree[tree['scale']==scale[i-1]]))

        for first, last, member_props in group_gen:
            result_III[first:last] = np.sum(member_props[0])
            result_II[first:last] = np.sum(member_props[1])
            result_metal[first:last] = np.sum(member_props[2])
            result_poll[first:last] = np.sum(member_props[3])
            result_enrich[first:last] = np.sum(member_props[4])
            result_DCBH[first:last] = np.sum(member_props[5])

        idx,idy = halotools.crossmatch(halos['desc_id'],tree['id'])
    
        tree['m_popIII'][idy] = result_III[idx]
        tree['m_popII'][idy] = result_II[idx]
        tree['M_metal'][idy] = result_metal[idx]
        tree['pristine'][idy] = result_poll[idx] 
        tree['polluted'][idy] = result_poll[idx]
        tree['enriched'][idy] = result_enrich[idx]
        tree['N_DCBH'][idy] = result_DCBH[idx]

        # self consistent LW calculation
        J21_int[i] =  J21_int_PopII[i] + J21_int_PopIII[i]
        
        ### PopIII calculation
        ## LW without self consistent
        M_LW[i] = 6.44*10.0**6.0 * (J21_ext[i])**0.457 * ((1.0/scale[i])/31.0)**-3.557
        ### LW with self consistent
        #M_LW[i] = 6.44*10.0**6.0 * (J21_ext[i] + J21_int[i])**0.457 * ((1.0/scale[i])/31.0)**-3.557
        M_PIII[i] = np.maximum(threshold.M_III(scale[i]),M_LW[i])
        M_PIII[i] = np.minimum(threshold.M_HI(scale[i]),M_PIII[i])

        mask_PIII = (tree['scale']==scale[i]) & (tree['mvir'] > M_PIII[i]) & (tree['mvir']>M_WDM) & (tree['polluted']==0) & (tree['enriched']==0)
        N_PopIII = parameter.Nfrag*(3.02*(tree['mvir'][mask_PIII]/10.0**6.0)**(2.0/3.0)*1.0/(tree['scale'][mask_PIII]*31.0))+1
        #N_PopIII = 1.0 + N_PopIII*0.0
        #mask = np.less(N_PopIII*parameter.star_popIII[imf],parameter.sfe_III[sfe]*constant.f_baryon*tree['mvir'][mask_PIII].quantity)
        #N_gas = np.trunc(parameter.sfe_III[sfe]*constant.f_baryon*np.array(tree['mvir'][mask_PIII])/parameter.star_popIII[imf])
        
        tree['enricher'][mask_PIII] = 3
        tree['m_popIII'][mask_PIII] = PopIII_IMF(parameter.star_popIII[imf],parameter.alpha[a],mask_PIII)
        tree['III_new'][mask_PIII] = tree['m_popIII'][mask_PIII]
        
        #### pop II calculation
        #M_PII[i] = threshold.M_HI(scale[i])
        M_PII[i] = np.minimum(threshold.M_HI(scale[i]),M_LW[i])
        mask_PII = (tree['scale']==scale[i]) & (tree['mvir'] > M_PII[i]) & (tree['mvir']>M_WDM) & ((tree['polluted']!=0) | (tree['enriched']!=0)) & (tree['III_new']==0.0)
        popII_new = parameter.sfe_II*(constant.f_baryon*tree['mvir'][mask_PII] - tree['m_popII'][mask_PII])
        popII_new[np.where(popII_new<0)]=0.0
        N_SN = popII_new*0
        metal = popII_new*0.0
        popII_LW = popII_new*0.0


        if len(popII_new)>0:
            for j in range(0,len(popII_new)):
                if popII_new[j]>0.0:
                    s = np.random.power(3.35,4*int(popII_new[j]))
                    mass = 0.08 * (1-s) ** (-1.0/(3.35-1.0))
                    cum = np.cumsum(mass)
                    stars = mass[np.where(cum<popII_new[j])]
                    metal[j] = np.sum(0.0075 * stars[np.where(stars>8.0)]**2.0 - 0.045*stars[np.where(stars>8.0)])
                    N_SN[j] = len(stars[np.where(stars>8.0)])
                    popII_new[j] = np.sum(stars[np.where(stars<8.0)])
                    popII_LW[j] = np.sum(stars[np.where(stars>8.0)])
    
        ### analytical treatment assuming a Salpeter
        #N_SN = np.trunc(0.0067*popII_new)
    
        tree['m_popII'][mask_PII] += popII_new
        tree['m_popII_LW'][mask_PII] = popII_LW
        tree['enricher'][mask_PII] = 2
        tree['num_SN'][mask_PII]=N_SN
        tree['E_SN'][mask_PII]=parameter.f_kinetic[fkin]*N_SN*10.0**51.0
        tree['M_metal'][mask_PII] += metal        
        
        tree['II_new'][mask_PII] = popII_new
        
        ### calculating the radii - in physical kpc
        tree['R_enrich'][tree['scale']==scale[i]] = Renrich(scale[i])
        tree['R_ejecta'][tree['scale']==scale[i]] = Rejecta(scale[i])
    
        ### calculate the enrichement
        mask_eject = (tree['scale']<scale[i]) & (tree['R_ejecta']>0.0)
        ejecter = tree[mask_eject]
        R_metal = ejecter['R_ejecta'].quantity*scale[i]/ejecter['scale']
        f_metal = np.zeros(len(R_metal))
        f_metal = ejecter['M_metal']/(constant.f_baryon * ejecter['mvir'] + constant.f_baryon*(4.0*math.pi/3.0)*10**-9.0*rho_IGM(scale[i-1])*(ejecter['R_ejecta']**3.0))
        for j in range (0,len(tree[mask_eject])):
            ### for the non periodic condition
            d = np.sqrt((tree['x(17)'] - ejecter['x(17)'][j])**2.0 + (tree['y(18)'] - ejecter['y(18)'][j])**2.0 + (tree['z(19)'] - ejecter['z(19)'][j])**2.0)
            mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
            tree['enriched'][mask_enrich] = 1 
            mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
            tree['III_enrich'][mask_enrich_3] = 1
            mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
            tree['II_enrich'][mask_enrich_2] = 1
                    
            ### periodic in the x direction    
            if ejecter['x(17)'][j]*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt(((tree['x(17)']-parameter.boxsize) - ejecter['x(17)'][j])**2.0 + (tree['y(18)'] - ejecter['y(18)'][j])**2.0 + (tree['z(19)'] - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1
   
            elif (parameter.boxsize-ejecter['x(17)'][j])*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt(((tree['x(17)']+parameter.boxsize) - ejecter['x(17)'][j])**2.0 + (tree['y(18)'] - ejecter['y(18)'][j])**2.0 + (tree['z(19)'] - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1

            ### periodic in the y direction    
            if ejecter['y(18)'][j]*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt((tree['x(17)'] - ejecter['x(17)'][j])**2.0 + ((tree['y(18)']-parameter.boxsize) - ejecter['y(18)'][j])**2.0 + (tree['z(19)'] - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1
                
            elif (parameter.boxsize-ejecter['y(18)'][j])*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt((tree['x(17)'] - ejecter['x(17)'][j])**2.0 + ((tree['y(18)']+parameter.boxsize) - ejecter['y(18)'][j])**2.0 + (tree['z(19)'] - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1
                
            ### periodic in the z direction    
            if ejecter['z(19)'][j]*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt((tree['x(17)'] - ejecter['x(17)'][j])**2.0 + (tree['y(18)'] - ejecter['y(18)'][j])**2.0 + ((tree['z(19)']-parameter.boxsize) - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1
                
            elif (parameter.boxsize-ejecter['z(19)'][j])*ejecter['scale'][j] < R_metal[j]:
                d = np.sqrt((tree['x(17)'] - ejecter['x(17)'][j])**2.0 + (tree['y(18)'] - ejecter['y(18)'][j])**2.0 + ((tree['z(19)']+parameter.boxsize) - ejecter['z(19)'][j])**2.0)
                mask_enrich = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (f_metal[j]>parameter.f_crit))
                tree['enriched'][mask_enrich] = 1 
                mask_enrich_3 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==3) & (f_metal[j]>parameter.f_crit))
                tree['III_enrich'][mask_enrich_3] = 1
                mask_enrich_2 = ((tree['scale']==scale[i]) & (R_metal[j]>1000.0*d*scale[i]) & (tree['enricher']==2) & (f_metal[j]>parameter.f_crit))
                tree['II_enrich'][mask_enrich_2] = 1
                
        ### setting the new polluted threshold
        tree['polluted'][(tree['scale']==scale[i]) & (tree['m_popIII'] > 0.0)] = 1
        tree['polluted'][(tree['scale']==scale[i]) & (tree['m_popII'] > 0.0)] = 1

        tree['enriched'][(tree['scale']==scale[i]) & (tree['m_popIII'] > 0.0)] = 1
        tree['enriched'][(tree['scale']==scale[i]) & (tree['m_popII'] > 0.0)] = 1
        
        print (scale[i])
        print (11.18/12.39 * scale[i])
        print("\n")

In [10]:
for imf in range(0,len(parameter.star_popIII)): 
    for a in range(0,len(parameter.alpha)):
        for fkin in range (0,len(parameter.f_kinetic)): 
            for sfe in range (0,len(parameter.sfe_III)): 
                for PISN in range (0,len(parameter.PISN_min)):
                    if parameter.star_popIII[imf]==40.0: 
                        parameter.sf_metal_III = 0.2 
                    else: 
                        if parameter.star_popIII[imf]==150.0: 
                            parameter.sf_metal_III = 0.7
            
                    tree = ascii.read('/Users/mbovill/Research/high_z/2M_512/tree_0_0_0.dat',delimiter=' ')
                    original = True

                    tree.rename_column('scale(0)','scale')
                    tree.rename_column('id(1)','id')
                    tree.rename_column('desc_scale(2)','desc_scale')
                    tree.rename_column('desc_id(3)','desc_id')
                    tree.rename_column('mvir(10)','mvir')
                    tree.rename_column('rvir(11)','rvir')
                    tree.rename_column('Tree_root_ID(29)','Tree_root_ID')
                    tree.rename_column('phantom(8)','phantom')

                    scale = np.unique(tree['scale'])
                    #J21_ext = externalJ21.J21_constant(scale,0.0)
                    J21_ext = externalJ21.TS09_low(scale)

                    J21_int = J21_ext*0.0

                    J21_int_PopIII = J21_ext*0.0
                    N_ph_III = J21_ext*0.0
                    nLW_PopIII = J21_ext*0.0

                    J21_int_PopII = J21_ext*0.0
                    N_ph_II = J21_ext*0.0
                    nLW_PopII = J21_ext*0.0

                    M_LW = scale*0.0
                    M_PIII = scale*0.0
                    M_PII = scale*0.0

                    R_ejecta = Table.Column(data=np.zeros(len(tree)),name='R_ejecta')
                    M_metal = Table.Column(data=np.zeros(len(tree)),name='M_metal')
                    M_metal_IGM = Table.Column(data=np.zeros(len(tree)),name='M_metal_IGM')
                    R_enrich = Table.Column(data=np.zeros(len(tree)),name='R_enrich')
                    E_SN = Table.Column(data=np.zeros(len(tree)),name='E_SN')
                    m_popIII = Table.Column(data=np.zeros(len(tree)),name='m_popIII')
                    m_popII = Table.Column(data=np.zeros(len(tree)),name='m_popII')
                    m_popII_LW = Table.Column(data=np.zeros(len(tree)),name='m_popII_LW')
                    num_SN = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='num_SN')
                    polluted = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='polluted')
                    pristine = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='pristine')
                    enriched = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='enriched')
                    III_enrich = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='III_enrich')
                    II_enrich = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='II_enrich')
                    enricher = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='enricher')    
                    III_new = Table.Column(data=np.zeros(len(tree),dtype=np.float),name='III_new')
                    N_III = Table.Column(data=np.zeros(len(tree),dtype=np.float),name='N_III')
                    II_new = Table.Column(data=np.zeros(len(tree),dtype=np.float),name='II_new')
                    N_DCBH = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='N_DCBH')
                    N_PISN = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='N_PISN')
                    N_CCSN = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='N_CCSN')

                    tree.add_column(enriched)
                    tree.add_column(E_SN)
                    tree.add_column(num_SN)
                    tree.add_column(m_popIII)
                    tree.add_column(m_popII)
                    tree.add_column(m_popII_LW)
                    tree.add_column(polluted)
                    tree.add_column(pristine)
                    tree.add_column(R_ejecta)
                    tree.add_column(R_enrich)
                    tree.add_column(M_metal)
                    tree.add_column(M_metal_IGM)
                    tree.add_column(III_enrich)
                    tree.add_column(II_enrich)
                    tree.add_column(enricher)
                    tree.add_column(III_new)
                    tree.add_column(N_III)
                    tree.add_column(II_new)
                    tree.add_column(N_DCBH)
                    tree.add_column(N_PISN)
                    tree.add_column(N_CCSN)

                    M_WDM = parameter.WDM_mass
                    metal_pollution_Nfrag()
                    print("\n")

                    LW_internal = Table([scale,J21_int_PopIII,J21_int_PopII],names = ('scale','J21_III','J21_II'))
            ##### output model information to a file
                    TS_model = '0000'
                    #if parameter.star_popIII[imf]==40.0:
                        #ascii.write(tree,'/Users/mbovill/Research/high_z/2M_512/models/TS09_low/TS09_low_CCSN_fkin%.0f_fIII%.0f_min1_max3.dat'%(10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)
                        #ascii.write(LW_internal,'/Users/msbovill/high_z/2M_512/models/J_%s/J21_internal_J_%s_SC_CCSN_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(TS_model,TS_model,10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)
                    #elif parameter.star_popIII[imf]==150.0:
                        #ascii.write(tree,'/Users/mbovill/Research/high_z/2M_512/models/TS09_low/TS09_low_PISN_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)
                        #ascii.write(LW_internal,'/Users/msbovill/high_z/2M_512/models/J_%s/J21_internal_J_%s_SC_PISN_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(TS_model,TS_model,10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)
                    if parameter.star_popIII[imf] == 'mf':
                        #ascii.write(tree,'/Users/mbovill/Research/high_z/2M_512/models/TS09_low/NIII/TS09_low_Parketal.dat',overwrite=True)

                        ascii.write(tree,'/Users/mbovill/Research/high_z/2M_512/models/TS09_low/TS09_low_a%d_fkin%.0f_fIII%.0f_min1_max2_3_34.dat'%(10.0*parameter.alpha[a],10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe]),overwrite=True)
                        #ascii.write(LW_internal,'/Users/msbovill/high_z/2M_512/models/J_%s/J21_internal_J_%s_SC_a%d_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(TS_model,TS_model,10.0*parameter.alpha[a],10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)                    
                    else:
                        if parameter.star_popIII[imf]==-1.0:
                            ascii.write(tree,'/Users/mbovill/Research/high_z/2M_512/models/J21_0000/TS09_high_1PISN_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)
                            #ascii.write(LW_internal,'/Users/msbovill/high_z/2M_512/models/J_%s/J21_internal_J_%s_SC_1PISN_fkin%.0f_fIII%.0f_Nfrag%d.dat'%(TS_model,TS_model,10.0*parameter.f_kinetic[fkin],1000*parameter.sfe_III[sfe],parameter.Nfrag[nfrag]),overwrite=True)


                        

/var/folders/g2/_t00swrd1p38glw01l5c8prdw9b803/T/ipykernel_53269/2960179957.py:50: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num_SN = Table.Column(data=np.zeros(len(tree),dtype=np.int),name='num_SN')
/var/folders/g2/_t00swrd1p38glw01l5c8prdw9b803/T/ipykernel_53269/2960179957.py:51: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pr

Pop III power law IMF
Pop III power law IMF
0.0379
0.03419870863599677


Pop III power law IMF
0.0398
0.03591315577078289


Pop III power law IMF
0.0418
0.03771783696529459


Pop III power law IMF
0.0437
0.03943228410008071


Pop III power law IMF
0.0455
0.04105649717514124


Pop III power law IMF
0.0472
0.042590476190476186


Pop III power law IMF
0.0489
0.04412445520581114


Pop III power law IMF
0.0505
0.045568200161420505


Pop III power law IMF
0.0522
0.04710217917675545


Pop III power law IMF
0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF
0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop III power law IMF
0.0662
0.059734947538337364


Pop III power law IMF
0.0678
0.06117869249394673


Pop

Pop III power law IMF
0.1378
0.12434253430185634


Pop III power law IMF
0.1389
0.12533510895883776


Pop III power law IMF
0.1399
0.12623744955609362


Pop III power law IMF
0.1408
0.1270495560936239




Pop III power law IMF
Pop III power law IMF
0.0379
0.03419870863599677


Pop III power law IMF
0.0398
0.03591315577078289


Pop III power law IMF
0.0418
0.03771783696529459


Pop III power law IMF
0.0437
0.03943228410008071


Pop III power law IMF
0.0455
0.04105649717514124


Pop III power law IMF
0.0472
0.042590476190476186


Pop III power law IMF
0.0489
0.04412445520581114


Pop III power law IMF
0.0505
0.045568200161420505


Pop III power law IMF
0.0522
0.04710217917675545


Pop III power law IMF
0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF


/var/folders/g2/_t00swrd1p38glw01l5c8prdw9b803/T/ipykernel_53269/2969233784.py:15: RuntimeWarning: invalid value encountered in power
  return R_en**(1.0/3.0)


0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop III power law IMF
0.0662
0.059734947538337364


Pop III power law IMF
0.0678
0.06117869249394673


Pop III power law IMF
0.0692
0.06244196933010492


Pop III power law IMF
0.0705
0.06361501210653753


Pop III power law IMF
0.0719
0.06487828894269572


Pop III power law IMF
0.0734
0.0662317998385795


Pop III power law IMF
0.0747
0.0674048426150121


Pop III power law IMF
0.0753
0.06794624697336563


Pop III power law IMF
0.0762
0.06875835351089589


Pop III power law IMF
0.0775
0.06993139628732849


Pop III power law IMF
0.0789
0.07119467312348668


Pop III power law IMF
0.0802
0.07236771589991928


Pop III power law IMF
0.0816
0.07363099273607748


Pop III power law IMF
0.0829
0.0748040355125101


Pop III power law IMF
0.0842
0.07597707828894269


Pop III power law IMF
0.0855
0.0771501210653753


Po

0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF
0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop III power law IMF
0.0662
0.059734947538337364


Pop III power law IMF
0.0678
0.06117869249394673


Pop III power law IMF
0.0692
0.06244196933010492


Pop III power law IMF
0.0705
0.06361501210653753


Pop III power law IMF
0.0719
0.06487828894269572


Pop III power law IMF
0.0734
0.0662317998385795


Pop III power law IMF
0.0747
0.0674048426150121


Pop III power law IMF
0.0753
0.06794624697336563


Pop III power law IMF
0.0762
0.06875835351089589


Pop III power law IMF
0.0775
0.06993139628732849


Pop III power law IMF
0.0789
0.07119467312348668


Pop III power law IMF
0.0802
0.07236771589991928




0.0472
0.042590476190476186


Pop III power law IMF
0.0489
0.04412445520581114


Pop III power law IMF
0.0505
0.045568200161420505


Pop III power law IMF
0.0522
0.04710217917675545


Pop III power law IMF
0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF
0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop III power law IMF
0.0662
0.059734947538337364


Pop III power law IMF
0.0678
0.06117869249394673


Pop III power law IMF
0.0692
0.06244196933010492


Pop III power law IMF
0.0705
0.06361501210653753


Pop III power law IMF
0.0719
0.06487828894269572


Pop III power law IMF
0.0734
0.0662317998385795


Pop III power law IMF
0.0747
0.0674048426150121


Pop III power law IMF
0.0753
0.06794624697336563


0.0398
0.03591315577078289


Pop III power law IMF
0.0418
0.03771783696529459


Pop III power law IMF
0.0437
0.03943228410008071


Pop III power law IMF
0.0455
0.04105649717514124


Pop III power law IMF
0.0472
0.042590476190476186


Pop III power law IMF
0.0489
0.04412445520581114


Pop III power law IMF
0.0505
0.045568200161420505


Pop III power law IMF
0.0522
0.04710217917675545


Pop III power law IMF
0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF
0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop III power law IMF
0.0662
0.059734947538337364


Pop III power law IMF
0.0678
0.06117869249394673


Pop III power law IMF
0.0692
0.06244196933010492


Pop III power law IMF
0.0705
0.0636150121065375

Pop III power law IMF
0.1399
0.12623744955609362


Pop III power law IMF
0.1408
0.1270495560936239




Pop III power law IMF
Pop III power law IMF
0.0379
0.03419870863599677


Pop III power law IMF
0.0398
0.03591315577078289


Pop III power law IMF
0.0418
0.03771783696529459


Pop III power law IMF
0.0437
0.03943228410008071


Pop III power law IMF
0.0455
0.04105649717514124


Pop III power law IMF
0.0472
0.042590476190476186


Pop III power law IMF
0.0489
0.04412445520581114


Pop III power law IMF
0.0505
0.045568200161420505


Pop III power law IMF
0.0522
0.04710217917675545


Pop III power law IMF
0.0539
0.0486361581920904


Pop III power law IMF
0.0556
0.05017013720742534


Pop III power law IMF
0.0571
0.051523648103309116


Pop III power law IMF
0.0587
0.05296739305891848


Pop III power law IMF
0.0602
0.054320903954802255


Pop III power law IMF
0.0617
0.055674414850686034


Pop III power law IMF
0.0633
0.05711815980629539


Pop III power law IMF
0.0647
0.058381436642453585


Pop

Pop III power law IMF
0.1358
0.12253785310734464


Pop III power law IMF
0.1368
0.12344019370460049


Pop III power law IMF
0.1378
0.12434253430185634


Pop III power law IMF
0.1389
0.12533510895883776


Pop III power law IMF
0.1399
0.12623744955609362


Pop III power law IMF
0.1408
0.1270495560936239






In [ ]:
z = np.linspace(4,50,1000)

z_III = 1.0/tree['scale'][tree['III_new']>0.0] - 1.0
M_PIII = tree['mvir'][tree['III_new']>0.0]
M_LW = 6.44*10**6.0 * (externalJ21.J21_constant(1.0/(z+1.0),0.001))**0.457 * ((z+1.0)/31.0)**-3.55
plt.plot(z,threshold.M_HI(1.0/(z+1.0)),c='black',ls=":")
plt.plot(z,threshold.M_III(1.0/(z+1.0)),c='black',ls="--")
plt.plot(z,M_LW,c='blue')
M_LW_tot = 6.44*10**6.0 * (J21_int+externalJ21.J21_constant(scale,0.001))**0.457 * (31.0*scale)**3.55
plt.plot(1.0/scale - 1.0,M_LW_tot,c='red')
plt.scatter(z_III,M_PIII,s=5,c='black')


#plt.colorbar(ax)
#plt.fill_between(1.0/sf - 1,threshold.M_HI(sf),10**11.0,facecolor='black',alpha=0.2)
#plt.fill_between(1.0/sf - 1,threshold.M_medium(sf),10**11.0,edgecolor='none',facecolor='cyan',alpha=0.2)
#plt.plot(1.0/sf - 1,threshold.M_III(sf),c='k')

plt.yscale('log')
plt.xlim(5,30)
plt.ylim(7*10**4.0,10**9.0)

plt.savefig("consistent_LW_test.pdf")
plt.show()

plt.plot(1.0/scale-1.0,externalJ21.J21_constant(scale,0.001))
plt.plot(1.0/scale-1.0,J21_int)
plt.yscale('log')
plt.savefig("consistent_LW_J21.pdf")

plt.show()

print(N_ph_III[N_ph_III>0.0])

plt.plot(1.0/scale-1.0,nLW_PopIII)
plt.yscale('log')
plt.show()

plt.show()
plt.plot(1.0/scale-1.0,J21_int)
plt.plot(1.0/scale-1.0,J21_int_PopIII)
plt.plot(1.0/scale-1.0,J21_int_PopII)
plt.yscale('log')
plt.show()

plt.scatter(1.0/scale,l)
plt.show()

In [ ]:
N_output = np.zeros(len(scale))
for i in range(0,len(scale)):
    N_output[i] = len(scale[(scale<=scale[i]) & (scale>11.18/12.39*scale[i])])

plt.plot(1.0/scale,N_output,'bo')
plt.show()

In [ ]:
plt.scatter(tree['mvir'][tree['III_new']>0.0],tree['R_ejecta'][tree['III_new']>0.0])
plt.xscale('log')
plt.yscale('log')
#plt.ylim(10**10,10**12)
plt.show()

plt.scatter(tree['III_new'][tree['III_new']>0.0],tree['R_ejecta'][tree['III_new']>0.0])
#plt.xscale('log')
#plt.yscale('log')
plt.show()

print(tree['R_ejecta'][tree['R_ejecta']>0.0])

In [ ]:
plt.scatter(tree['E_SN'][tree['III_new']>0.0]/10**51.0,tree['N_PISN'][tree['III_new']>0.0]*0.3*10.0**53.0/10**51.0)
plt.xlim(0.1,1000)
plt.ylim(0.1,1000)
plt.xscale('log')
plt.yscale('log')


In [ ]:
print(np.log10(10.0**53.0))